<a href="https://colab.research.google.com/github/hehuan2363/CanadaInnovationProposal/blob/main/testQ%26AforABook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain openai
!pip install unstructured[local-inference]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.2 MB/s eta 0:00:00


In [2]:
from getpass import getpass

OPENAI_API_KEY=getpass("OpenAI API Key: ")


OpenAI API Key: ··········


In [4]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
#loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [6]:
loader = UnstructuredPDFLoader(root_dir+ "field-guide-to-data-science.pdf")

In [7]:
root_dir+ "field-guide-to-data-science.pdf"

'/content/gdrive/My Drive/field-guide-to-data-science.pdf'

In [8]:
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 201014 characters in your document


In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

In [11]:
texts = text_splitter.split_documents(data)

In [12]:
print (f'Now you have {len(texts)} documents')

Now you have 240 documents


Creat embedding now


In [13]:
!pip install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 15.6 MB/s eta 0:00:00


In [14]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/usr/local/lib/python3.9/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [15]:
PINECONE_API_KEY = getpass("PINECONE_API_KEY: ")


PINECONE_API_KEY: ··········


In [16]:
PINECONE_API_ENV = 'us-east1-gcp'

In [17]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "testlangchain"

In [18]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [21]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [22]:
query = "What are examples of good data science teams?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [23]:
docs

[Document(page_content='Steven Mills (@stevndmills)Data Science truly can change the world.Stephanie Beben(@boozallen)Begin every new data challenge with deep curiosity along with a healthy dose of skepticism.Alex Cosmas (@boozallen)Data scientists should be truth-seekers, not fact-seekers.Kirk Borne(@KirkDBorne)Focus on value, not volume.Brian Keller (@boozallen)Grit will get you farther than talent.Drew Farris (@drewfarris)Don’t forget to play. Play with tools, play with data, and play with algorithms. You just might discover something that will help you solve that next nagging problem. 13Meet Your Guides', metadata={}),
 Document(page_content='Data Science teams.\n\n›› Data Science capabilities can be built over time.\n\nOrganizations mature through a series of stages – Collect,\n\nDescribe, Discover, Predict, Advise – as they move from data\n\ndeluge to full Data Science maturity. At each stage, they can\n\ntackle increasingly complex analytic goals with a wider breadth\n\nof analy

In [26]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [27]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [28]:
query = "What is the collect stage of data maturity?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [29]:
chain.run(input_documents=docs, question=query)

' The collect stage of data maturity focuses on collecting internal or external datasets. An example of this stage is gathering sales records and corresponding weather data.'

In [30]:
query = "What are examples of good data science teams?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [31]:
chain.run(input_documents=docs, question=query)

' Good data science teams should have a broad view of the organization, foster trust and transparent communication, provide a path into the data science "club" and empower employees to feel comfortable and conversant with data science. They should also have qualities such as deep curiosity, healthy skepticism, truth-seeking, focus on value, grit, and playfulness.'

In [32]:
import sys
print(sys.modules.keys())

dict_keys(['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', 'encodings.latin_1', '_abc', 'abc', 'io', '__main__', 'types', 'enum', '_sre', 'sre_constants', 'sre_parse', 'sre_compile', '_collections_abc', '_heapq', 'heapq', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections', '_functools', 'functools', '_locale', 'copyreg', 're', 'warnings', '_stat', 'stat', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_bootlocale', 'importlib._bootstrap', 'importlib._bootstrap_external', 'importlib', 'importlib.machinery', 'collections.abc', 'contextlib', 'typing.io', 'typing.re', 'typing', 'importlib.abc', 'importlib.util', 'google', 'google.cloud', 'google.logging', 'mpl_toolkits', 'sphinxcontrib', 'sitecustomize', 'site', '_weakrefset', 'weakref', 'p